In [ ]:
import pandas as pd
import numpy as np
import os

In [ ]:
import kagglehub
# Download latest version
path = kagglehub.dataset_download("ulrikthygepedersen/speed-dating")
print("Path to dataset files:", path)

In [ ]:
print(os.listdir(path))

In [ ]:
df = pd.read_csv(path+'/speeddating.csv')
df.loc[:, (df.dtypes == 'object').values] = df.loc[:, (df.dtypes == 'object').values].map(lambda x: x.replace("b'", '').replace("'", ""))

In [ ]:
summary_df = pd.DataFrame({
    'dtype': df.dtypes.astype(str),
    'unique_count': df.nunique(),
    'missing_count': df.isnull().sum(),
    'mean': df.mean(numeric_only=True),
    'std': df.std(numeric_only=True),
    'min': df.min(numeric_only=True),
    'max': df.max(numeric_only=True),
    'mode': df.mode().iloc[0],  # 최빈값
    'zero_count': (df == 0).sum()  # 0값 개수
})

In [ ]:
pd.set_option('display.max_rows', 1000)   
pd.set_option('display.max_columns', 2000) 
summary_df = summary_df.T

In [ ]:
type(summary_df)

In [ ]:
summary_df[summary_df.loc['dtype',:] == 'object']

In [ ]:
type(summary_df.loc['dtype',:] == 'object')

In [ ]:
print(summary_df.index.tolist())

In [ ]:
summary_df.loc['dtype'] == 'object'

In [ ]:
summary_df.loc[:, summary_df.loc['dtype'] == 'object']

In [ ]:
need_encod = summary_df.loc[:, (summary_df.loc['unique_count'] < 10) & (summary_df.loc['dtype'] == 'object')].columns

In [ ]:
need_encod.shape

In [ ]:
df_encoded = pd.get_dummies(df, prefix=None, prefix_sep='_', dummy_na=True, columns=need_encod, drop_first=False)

In [ ]:
df_encoded.shape

In [ ]:
corr_matrix = df_encoded.corr()
print(corr_matrix)


In [ ]:
df_encoded.dtypes == 'object'

In [ ]:
summary_df_endcoded = pd.DataFrame({
    'dtype': df_encoded.dtypes.astype(str),
    'unique_count': df_encoded.nunique(),
    'missing_count': df_encoded.isnull().sum(),
    'mean': df_encoded.mean(numeric_only=True),
    'std': df_encoded.std(numeric_only=True),
    'min': df_encoded.min(numeric_only=True),
    'max': df_encoded.max(numeric_only=True),
    'mode': df_encoded.mode().iloc[0],  # 최빈값
    'zero_count': (df_encoded == 0).sum()  # 0값 개수
})

In [ ]:
summary_df_endcoded[(summary_df_endcoded.loc[:,'dtype'] == 'object') & (summary_df_endcoded.loc[:,'unique_count'] > 10)]

In [ ]:
df_encoded['field'].unique()

In [ ]:
def categorize_field(field):
    field_lower = str(field).lower()
    law_keywords = ['law', 'legal']
    business_keywords = ['business', 'finance', 'mba', 'marketing', 'management', 'consulting', 'real estate', 'private equity']
    science_keywords = ['engineering', 'biology', 'chemistry', 'physics', 'math', 'medicine', 'biomedical', 'neuroscience', 'statistics', 'informatics', 'environmental', 'ecology', 'genetics', 'microbiology', 'biotechnology', 'industrial', 'operations research']
    social_keywords = ['education', 'social work', 'psychology', 'public administration', 'political science', 'international affairs', 'sociology', 'anthropology', 'public health', 'health policy', 'human rights', 'history', 'communication', 'curriculum', 'teaching', 'school', 'counseling', 'nutrition', 'speech', 'policy', 'development', 'urban planning']
    arts_keywords = ['art', 'music', 'film', 'theater', 'philosophy', 'literature', 'creative writing', 'language', 'nonfiction', 'poetry', 'acting', 'drama', 'classics']

    if any(k in field_lower for k in law_keywords):
        return 'Law'
    elif any(k in field_lower for k in business_keywords):
        return 'Business/Finance'
    elif any(k in field_lower for k in science_keywords):
        return 'Science/Engineering'
    elif any(k in field_lower for k in social_keywords):
        return 'Social Sciences/Education'
    elif any(k in field_lower for k in arts_keywords):
        return 'Arts/Humanities/Other'
    else:
        return 'Other'

df_encoded['category'] = df_encoded['field'].apply(categorize_field)


In [ ]:
df_encoded['category'].unique()

In [ ]:
df_encoded = pd.get_dummies(df_encoded, prefix=None, prefix_sep='_', dummy_na=True, columns=['category'], drop_first=False)

In [ ]:
df_encoded.shape

In [ ]:
mask = (df_encoded.dtypes == 'object') & (df_encoded.nunique() >= 10)
df_encoded.loc[:, mask].columns

In [ ]:
df_encoded.shape

In [ ]:
df_encoded = df_encoded.drop(['field'], axis=1)

In [ ]:
mask = (df_encoded.dtypes == 'object')
df_encoded.loc[:, mask].columns

In [ ]:
corr_matrix = df_encoded.corr()
print(corr_matrix)

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# 히트맵 시각화
plt.figure(figsize=(20, 15))  # 크기 조절
sns.heatmap(corr_matrix, cmap='coolwarm', center=0, annot=False, fmt='.2f', 
            cbar=True, square=True, linewidths=0.5)
plt.title('Correlation Matrix Heatmap')
plt.show()


In [ ]:
df['match']